### Check if you are able to get response from Claude through AWS Bedrock

In [14]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_claude",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

Hi!


In [15]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_titan",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Hello!


### Check if the API docs are fetched correctly

In [ ]:
import requests
weather_response = requests.get("http://localhost:8000/weather")

print(weather_response.json()["response"])

### Check if the user query is fetched correctly

In [2]:
import requests
weather_response = requests.post("http://localhost:8000/user_query", json={"user_query": "What is the weather in London?"})

print(weather_response.json())

{'response': 'What is the weather in London?'}


### Test the query with Claude as both the mother as well as the child agent

In [7]:
# First get user query
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "How is the weather in Nagpur Like?"})
print(query_response.json())
# Then create child agent
agent_response = requests.get("http://localhost:8000/claude_child_agent", 
                            params={"user_query": query_response.json()["response"]})

{'response': 'How is the weather in Nagpur Like?'}


### Test the query with Claude as the mother agent and Titan as the child agent

In [ ]:
# Modified client code
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "How is the weather in Saulo Paulo Like?"})
print("Query Response:", query_response.json())

agent_response = requests.get("http://localhost:8000/titan_child_agent", 
                            params={"user_query": query_response.json()["response"]})
print("Agent Response:", agent_response.json())

Query Response: {'response': 'How is the climate in Delhi Like?'}
Agent Response: {'message': 'Child agent successfully created', 'response': 'import boto3\nimport requests\nimport json\nimport os\nfrom dotenv import load_dotenv\n\nload_dotenv()\napi_key = os.getenv("OPENWEATHERMAP_API_KEY")\nbedrock_client = boto3.client(\'bedrock-runtime\')\n\ndef call_titan(prompt: str):\n    payload = {\n        "inputText": prompt,\n        "textGenerationConfig": {\n            "maxTokenCount": 200,\n            "stopSequences": [],\n            "temperature": 0.3,\n            "topP": 0.4\n        }\n    }\n\n    response = bedrock_client.invoke_model(\n        body=json.dumps(payload),\n        modelId="amazon.titan-text-express-v1",\n        contentType="application/json",\n        accept="application/json"\n    )\n    \n    response_body = json.loads(response[\'body\'].read())\n    return response_body[\'results\'][0][\'outputText\']\n\ndef get_weather(city_name: str):\n    api_endpoint = "ht